# OCSVM Baseline Model
*Created by Holger Buech, Q1/2019*

**Description**   

Reimplemenation of a OCSVM approach to Continuous Authentication described by [1]. Used as baseline model for futher experiments.

**Purpose**

- Get basic idea about authentication performance using raw data
- Verify results of [1]

**Data Sources**   

- [H-MOG Dataset](http://www.cs.wm.edu/~qyang/hmog.html)  
  (Downloaded beforehand using  [./src/data/make_dataset.py](./src/data/make_dataset.py), stored in [./data/external/hmog_dataset/](./data/external/hmog_dataset/) and converted to [./data/processed/hmog_dataset.hdf5](./data/processed/hmog_dataset.hdf5))

**References**   

[1] Centeno, M. P. et al. (2018): Mobile Based Continuous Authentication Using Deep Features. Proceedings of the 2^nd International Workshop on Embedded and Mobile Deep Learning (EMDL), 2018, 19-24.

**Table of Contents**

**1 - [Preparations](#1)**  
1.1 - [Imports](#1.1)  
1.2 - [Configuration](#1.2)  
1.3 - [Experiment Parameters](#1.3)  
1.4 - [Select Approach](#1.4)  

**2 - [Data Preparations](#2)**  
2.1 - [Load Dataset](#2.1)  
2.2 - [Normalize Features (if global)](#2.2)  
2.3 - [Split Dataset for Valid/Test](#2.3)  
2.4 - [Check Splits](#2.4)  
2.5 - [Reshape Features](#2.5)  

**3 - [Hyperparameter Optimization](#3)**  
3.1 - [Load cached Data](#3.1)   
3.2 - [Search for Parameters](#3.2)   
3.3 - [Inspect Search Results](#3.3)  

**4 - [Testing](#4)**   
4.1 - [Load cached Data](#4.1)    
4.2 - [Evaluate Authentication Performance](#4.2)  
4.3 - [Evaluate increasing Training Set Size (Training Delay)](#4.3)  
4.4 - [Evaluate increasing Test Set Size (Detection Delay)](#4.4)  

## 1. Preparations <a id='1'>&nbsp;</a> 

### 1.1 Imports <a id='1.1'>&nbsp;</a> 

# restore session

In [1]:
!ls


chapter-4-5-1-ocsvm-parameter-demo.ipynb	 NOPE WE TRAINING
chapter-5-2-1-exploration-hmog-statistics.ipynb  output
chapter-5-4-ocsvm.db				 thesis
chapter-5-4-ocsvm.ipynb				 thesis-env
chapter-5-5-siamese-cnn.ipynb			 utils.ipynb
chapter-5-6-4-explore-normalization.ipynb	 VAEs - AD .ipynb
chapter-5-8-final-comparison.ipynb		 VAEs.db
my_path						 venv


In [27]:
import dill
#dill.load_session('chapter-5-4-ocsvm.db')
dill.dump_session('chapter-5-4-ocsvm.db')

In [2]:
#! virtualenv --python=/usr/bin/python3 venv
!source venv/bin/activate

In [11]:
import scipy as tf

print(tf.__version__)

1.5.2


In [14]:
!python3 -m pip uninstall scipy --y
!python3 -m pip uninstall numpy --y
!python3 -m pip install -I numpy==1.16.4
!python3 -m pip install -I scipy==1.2.1

Found existing installation: scipy 1.5.2
ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/pip/_internal/cli/base_command.py", line 216, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.6/site-packages/pip/_internal/commands/uninstall.py", line 90, in run
    auto_confirm=options.yes, verbose=self.verbosity > 0,
  File "/usr/local/lib/python3.6/site-packages/pip/_internal/req/req_install.py", line 684, in uninstall
    uninstalled_pathset = UninstallPathSet.from_dist(dist)
  File "/usr/local/lib/python3.6/site-packages/pip/_internal/req/req_uninstall.py", line 535, in from_dist
    for path in uninstallation_paths(dist):
  File "/usr/local/lib/python3.6/site-packages/pip/_internal/req/req_uninstall.py", line 67, in unique
    for item in fn(*args, **kw):
  File "/usr/local/lib/python3.6/site-packages/pip/_internal/req/req_uninstall.py", line 85, in uninstallation_paths
    r = csv.reader(FakeFile(dist.get_met

In [8]:
# Standard
from pathlib import Path
import os
import sys
import dataclasses
import math
import warnings

# Extra
import pandas as pd
import numpy as np
from sklearn.svm import OneClassSVM
from sklearn.model_selection import cross_validate, RandomizedSearchCV
import statsmodels.stats.api as sms
from tqdm.auto import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

# Custom `DatasetLoader`class for easier loading and subsetting data from the datasets.
module_path = os.path.abspath(os.path.join(".."))  # supposed to be parent folder
if module_path not in sys.path:
    sys.path.append(module_path)
from src.utility.dataset_loader_hdf5 import DatasetLoader

# Global utitlity functions are in separate notebook
%run utils.ipynb

ImportError: cannot import name 'comb'

### 1.2 Configuration <a id='1.2'>&nbsp;</a>

In [ ]:
# Various Settings
SEED = 712  # Used for every random function
HMOG_HDF5 = Path.cwd().parent / "data" / "processed" / "hmog_dataset.hdf5"
EXCLUDE_COLS = ["sys_time"]
CORES = -1

# For plots and CSVs
OUTPUT_PATH = Path.cwd() / "output" / "chapter-6-1-3-ocsvm"
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)
REPORT_PATH = Path.cwd().parent / "reports" / "figures" # Figures for thesis

# Plotting
%matplotlib inline
utils_set_output_style()

In [ ]:
# Workaround to remove ugly spacing between progress bars
HTML("<style>.p-Widget.jp-OutputPrompt.jp-OutputArea-prompt:empty{padding: 0;border: 0;} div.output_subarea{padding:0;}</style>")

### 1.3 Experiment Parameters <a id='1.3'>&nbsp;</a> 
Selection of parameters set that had been tested in this notebook. Select one of them to reproduce results.

In [3]:
@dataclasses.dataclass
class ExperimentParameters:
    """Contains all relevant parameters to run an experiment."""

    name: str  # Name of Parameter set. Used as identifier for charts etc.
    frequency: int
    max_subjects: int
    max_test_subjects: int
    seconds_per_subject_train: float
    seconds_per_subject_test: float
    task_types: list  # Limit scenarios to [1, 3, 5] for sitting or [2, 4, 6] for walking, or don't limit (None)
    window_size: int  # After resampling
    step_width: int  # After resampling
    scaler: str  # {"std", "robust", "minmax"}
    scaler_scope: str  # {"subject", "session"}
    scaler_global: bool  # fit transform scale on all data (True) or fit on training only (False)
    ocsvm_nu: float  # Best value found in random search, used for final model
    ocsvm_gamma: float  # Best value found in random search, used for final model
    feature_cols: list  # Columns used as features
    exclude_subjects: list  # Don't load data from those users
        
    # Calculated values
    def __post_init__(self):
        # HDF key of table:
        self.table_name = f"sensors_{self.frequency}hz"

        # Number of samples per _session_ used for training:
        self.samples_per_subject_train = math.ceil(
            (self.seconds_per_subject_train * 100)
            / (100 / self.frequency)
            / self.window_size
        )

        # Number of samples per _session_ used for testing:
        self.samples_per_subject_test = math.ceil(
            (self.seconds_per_subject_test * 100)
            / (100 / self.frequency)
            / self.window_size
        )

        

# INSTANCES
# ===========================================================

# NAIVE_APPROACH
# -----------------------------------------------------------
NAIVE_MINMAX_OCSVM = ExperimentParameters(
    name="NAIVE-MINMAX_OCSVM",
    frequency=100,
    max_subjects= 40, # 90,
    max_test_subjects=15,# 3,
    seconds_per_subject_train=67.5,
    seconds_per_subject_test=67.5,    
    task_types=None,
    window_size=50,
    step_width=50,
    scaler="minmax",
    scaler_scope="subject",
    scaler_global=True,
    ocsvm_nu=0.086,
    ocsvm_gamma=0.091,
    feature_cols=[
        "acc_x",
        "acc_y",
        "acc_z",
        "gyr_x",
        "gyr_y",
        "gyr_z",
        "mag_x",
        "mag_y",
        "mag_z",
    ],
    exclude_subjects=[
        "733162",  # No 24 sessions
        "526319",  # ^
        "796581",  # ^
        "539502",  # Least amount of sensor values
        "219303",  # ^
        "737973",  # ^
        "986737",  # ^
        "256487",  # Most amount of sensor values
        "389015",  # ^
        "856401",  # ^
    ],
)

# VALID_APPROACH
# -----------------------------------------------------------
VALID_MINMAX_OCSVM = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-MINMAX-OCSVM",
    scaler_global=False,
    ocsvm_nu=0.165,
    ocsvm_gamma=0.039,
)

# NAIVE_ROBUST_APPROACH
# -----------------------------------------------------------
NAIVE_ROBUST_OCSVM = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="NAIVE-ROBUST-OCSVM",
    scaler="robust",
    scaler_global=True,
    ocsvm_nu=0.153,
    ocsvm_gamma=0.091,  # below median, selected by chart
)

# ROBUST_APPROACH (VALID)
# -----------------------------------------------------------
VALID_ROBUST_OCSVM = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="robust",
    scaler_global=False,
    ocsvm_nu=0.098,
    ocsvm_gamma=0.003,
)

### 1.4 Select approach <a id='1.4'>&nbsp;</a> 
Select the parameters to use for current notebook execution here!

In [5]:
P = VALID_ROBUST_OCSVM

**Overview of current Experiment Parameters:**

In [7]:
utils_ppp(P)

NameError: name 'utils_ppp' is not defined

## 2. Data Preparation <a id='2'>&nbsp;</a> 

### 2.1 Load Dataset <a id='2.1'>&nbsp;</a> 

In [ ]:
hmog = DatasetLoader(
    hdf5_file=HMOG_HDF5,
    table_name=P.table_name,
    max_subjects=P.max_subjects,
    task_types=P.task_types,
    exclude_subjects=P.exclude_subjects,
    exclude_cols=EXCLUDE_COLS,
    seed=SEED,
)

hmog.data_summary()

### 2.2 Normalize features (if global) <a id='2.2'>&nbsp;</a> 
Used here for naive approach (before splitting into test and training sets). Otherwise it's used during generate_pairs() and respects train vs. test borders.

In [ ]:
if P.scaler_global:
    print("Normalize all data before splitting into train and test sets...")
    hmog.all, _ = utils_custom_scale(
        hmog.all,
        scale_cols=P.feature_cols,        
        feature_cols=P.feature_cols,
        scaler_name=P.scaler,
        scope=P.scaler_scope,
        plot=True,
    )
else:
    print("Skipped, normalize after splitting.")

### 2.3 Split Dataset for Valid/Test <a id='2.3'>&nbsp;</a> 
In two splits: one used during hyperparameter optimization, and one used during testing.

The split is done along the subjects: All sessions of a single subject will either be in the validation split or in the testing split, never in both.

In [ ]:
hmog.split_train_test(n_test_subjects=P.max_test_subjects)
hmog.data_summary()

### 2.4 Check Splits <a id='2.4'>&nbsp;</a> 

In [ ]:
utils_split_report(hmog.train)

In [ ]:
utils_split_report(hmog.test)

### 2.5 Reshape Features  <a id='2.5'>&nbsp;</a> 

**Reshape & store Set for Validation:**

In [ ]:
df_train_valid = utils_reshape_features(
    hmog.train,
    feature_cols=P.feature_cols,
    window_size=P.window_size,
    step_width=P.step_width,
)

# Clean memory
del hmog.train
# %reset_selective -f hmog.train

print("Validation data after reshaping:")
display(df_train_valid.head())

# Store iterim data
df_train_valid.to_msgpack(OUTPUT_PATH / "df_train_valid.msg")

# Clean memory
# %reset_selective -f df_train_valid

**Reshape & store Set for Testing:**

In [ ]:
df_train_test = utils_reshape_features(
    hmog.test,
    feature_cols=P.feature_cols,
    window_size=P.window_size,
    step_width=P.step_width,
)

del hmog.test
# %reset_selective -f hmog.test

print("Testing data after reshaping:")
display(df_train_test.head())

# Store iterim data
df_train_test.to_msgpack(OUTPUT_PATH / "df_train_test.msg")

# Clean memory
# %reset_selective -f df_train_test

In [ ]:
# Clean Memory
# %reset_selective -f df_

## 3. Hyperparameter Optimization  <a id='3'>&nbsp;</a> 

### 3.1 Load cached Data <a id='3.1'>&nbsp;</a> 
Only the split dedicated for hyperparameter optimization is loaded

In [ ]:
df_train_valid = pd.read_msgpack(OUTPUT_PATH / "df_train_valid.msg")
df_train_valid.head()

### 3.2 Search for Parameters <a id='3.2'>&nbsp;</a> 

In [ ]:
param_dist = {"gamma": np.logspace(-3, 3), "nu": np.linspace(0.0001, 0.3)}
warnings.filterwarnings("ignore")
df_results = None  # Will be filled with randomsearch scores
for run in tqdm(range(3)):
    for df_cv_scenarios, owner, impostors in tqdm(
        utils_generate_cv_scenarios(
            df_train_valid,
            samples_per_subject_train=P.samples_per_subject_train,
            samples_per_subject_test=P.samples_per_subject_test,
            seed=SEED + run,
            scaler=P.scaler,
            scaler_global=P.scaler_global,
            scaler_scope=P.scaler_scope,
            feature_cols=P.feature_cols,
        ),
        desc="Owner",
        total=df_train_valid["subject"].nunique(),
        leave=False,
    ):
        X = np.array(df_cv_scenarios["X"].values.tolist())
        X = X.reshape(X.shape[-3], -1)  # flatten windows
        y = df_cv_scenarios["label"].values
        train_valid_cv = utils_create_cv_splits(df_cv_scenarios["mask"].values, SEED)
        model = OneClassSVM(kernel="rbf")

        random_search = RandomizedSearchCV(
            model,
            param_distributions=param_dist,
            cv=train_valid_cv,
            n_iter=80,
            n_jobs=CORES,
            refit=False,
            scoring={"eer": utils_eer_scorer, "accuracy": "accuracy"},
            verbose=0,
            return_train_score=False,
            iid=False,
            random_state=SEED,
        )
        random_search.fit(X, y)
        df_report = utils_cv_report(random_search, owner, impostors)
        df_report["run"] = run
        df_results = pd.concat([df_results, df_report], sort=False)

df_results.to_csv(OUTPUT_PATH / f"{P.name}_random_search_results.csv", index=False)

### 3.3 Inspect Search Results <a id='3.3'>&nbsp;</a> 
**Raw Results & Stats:**

In [ ]:
df_results = pd.read_csv(OUTPUT_PATH / f"{P.name}_random_search_results.csv")
print("Example from result table (head):")
display(
    df_results[df_results["rank_test_eer"] == 1]
    .sort_values("mean_test_eer")
    .head(10)
)
print("\n\n\nMost relevant statistics:")
display(
    df_results[df_results["rank_test_eer"] == 1][
        [
            "mean_fit_time",
            "param_nu",
            "param_gamma",
            "mean_test_accuracy",
            "std_test_accuracy",
            "mean_test_eer",
            "std_test_eer",
        ]
    ].describe()
)

**Plot parameters of top n of 30 results for every Owner:**

In [ ]:
utils_plot_randomsearch_results(df_results, n_top=1)
utils_save_plot(plt, REPORT_PATH / f"buech2019-ocsvm-{P.name.lower()}-parameters.pdf")

**Note:** Using median to select the best parameters, as mean is strongly influenced by outliers.

In [ ]:
# Clean Memory
# %reset_selective -f df_

## 4. Testing <a id='4'>&nbsp;</a> 

### 4.1 Load cached Data <a id='4.1'>&nbsp;</a> 
During testing, a split with different users than used for hyperparameter optimization is used:

In [ ]:
df_train_test = pd.read_msgpack(OUTPUT_PATH / "df_train_test.msg")

### 4.2 Evaluate Authentication Performance <a id='4.2'>&nbsp;</a> 

- Using Testing Split, Scenario Cross Validation, and multiple runs to lower impact of random session/sample selection.

In [ ]:
df_results = None  # Will be filled with cv scores

for i in tqdm(range(5), desc="Run", leave=False):  # Run whole test 5 times
    for df_cv_scenarios, owner, impostors in tqdm(
        utils_generate_cv_scenarios(
            df_train_test,
            samples_per_subject_train=P.samples_per_subject_train,
            samples_per_subject_test=P.samples_per_subject_test,
            seed=SEED + i,  # Change seed for different runs
            scaler=P.scaler,
            scaler_global=P.scaler_global,
            scaler_scope=P.scaler_scope,
            feature_cols=P.feature_cols,
        ),
        desc="Owner",
        total=df_train_test["subject"].nunique(),
        leave=False,
    ):
        X = np.array(df_cv_scenarios["X"].values.tolist())
        X = X.reshape(X.shape[-3], -1)  # flatten windows
        y = df_cv_scenarios["label"].values
        train_test_cv = utils_create_cv_splits(df_cv_scenarios["mask"].values, SEED)

        model = OneClassSVM(kernel="rbf", nu=P.ocsvm_nu, gamma=P.ocsvm_gamma)

        scores = cross_validate(
            model,
            X,
            y,
            cv=train_test_cv,
            scoring={
                "eer": utils_eer_scorer,
                "accuracy": "accuracy",
                "precision": "precision",
                "recall": "recall",
            },
            n_jobs=CORES,
            verbose=0,
            return_train_score=True,
        )
        df_score = pd.DataFrame(scores)
        df_score["owner"] = owner
        df_score["train_eer"] = df_score["train_eer"].abs()  # Revert scorer's signflip
        df_score["test_eer"] = df_score["test_eer"].abs()
        df_results = pd.concat([df_results, df_score], axis=0)

df_results.to_csv(OUTPUT_PATH / f"{P.name}_test_results.csv", index=False)

df_results.head()

**Load Results from "EER & Accuracy" evaluation & prepare for plotting:**

In [ ]:
df_results = pd.read_csv(OUTPUT_PATH / f"{P.name}_test_results.csv")
df_plot = df_results.rename(
    columns={"test_accuracy": "Test Accuracy", "test_eer": "Test EER", "owner": "Owner"}
).astype({"Owner": str})

**Plot Distribution of Accuracy per subject:**

In [ ]:
fig = utils_plot_acc_eer_dist(df_plot, "Test Accuracy")
utils_save_plot(plt, REPORT_PATH / f"buech2019-ocsvm-{P.name.lower()}-acc.pdf")

In [ ]:
fig = utils_plot_acc_eer_dist(df_plot, "Test EER")
utils_save_plot(plt, REPORT_PATH / f"buech2019-ocsvm-{P.name.lower()}-eer.pdf")

### 4.3 Evaluate increasing Training Set Size (Training Delay)<a id='4.3'>&nbsp;</a> 

- Testing different amounts of samples in training set
- Using Testing Split, Scenario Cross Validation, and multiple runs to lower impact of random session/sample selection.

In [ ]:
training_set_sizes = [2, 4, 6, 8, 20, 60, 120, 180, 250, 350, 500, 750]

df_results = None  # Will be filled with cv scores
for i in tqdm(range(5), desc="Run", leave=False):
    for n_train_samples in tqdm(training_set_sizes, desc="Train Size", leave=False):
        for df_cv_scenarios, owner, impostors in tqdm(
            utils_generate_cv_scenarios(
                df_train_test,
                samples_per_subject_train=P.samples_per_subject_train,
                samples_per_subject_test=P.samples_per_subject_test,
                limit_train_samples=n_train_samples,  # samples overall
                seed=SEED + i,  # Change seed for different runs
                scaler=P.scaler,
                scaler_global=P.scaler_global,
                scaler_scope=P.scaler_scope,
                feature_cols=P.feature_cols,
            ),
            desc="Owner",
            total=df_train_test["subject"].nunique(),
            leave=False,
        ):
            X = np.array(df_cv_scenarios["X"].values.tolist())
            X = X.reshape(X.shape[-3], -1)  # flatten windows
            y = df_cv_scenarios["label"].values
            train_test_cv = utils_create_cv_splits(df_cv_scenarios["mask"].values, SEED)

            model = OneClassSVM(kernel="rbf", nu=P.ocsvm_nu, gamma=P.ocsvm_gamma)

            scores = cross_validate(
                model,
                X,
                y,
                cv=train_test_cv,
                scoring={"eer": utils_eer_scorer},
                n_jobs=CORES,
                verbose=0,
                return_train_score=True,
            )
            df_score = pd.DataFrame(scores)
            df_score["owner"] = owner
            df_score["train_samples"] = n_train_samples
            df_score["train_eer"] = df_score[
                "train_eer"
            ].abs()  # Revert scorer's signflip
            df_score["test_eer"] = df_score["test_eer"].abs()
            df_results = pd.concat([df_results, df_score], axis=0)

df_results.to_csv(OUTPUT_PATH / f"{P.name}_train_delay_results.csv", index=False)
df_results.head()

**Load Results from "Training set size" evaluation & prepare for plotting:**

In [ ]:
df_results = pd.read_csv(OUTPUT_PATH / f"{P.name}_train_delay_results.csv")
df_plot = (
    df_results[["test_eer", "owner", "train_samples"]]
    .groupby(["owner", "train_samples"], as_index=False)
    .mean()
    .astype({"owner": "category"})
    .rename(
        columns={
            "test_eer": "Test EER",
            "owner": "Owner",
        }
    )
)
df_plot["Training Data in Seconds"] = df_plot["train_samples"] * P.window_size / P.frequency

**Plot EER with increasing number of training samples:**

In [ ]:
utils_plot_training_delay(df_plot)
utils_save_plot(plt, REPORT_PATH / f"buech2019-ocsvm-{P.name.lower()}-train-size.pdf")

### 4.4 Evaluate increasing Test Set Sizes (Detection Delay)<a id='4.4'>&nbsp;</a> 

In [ ]:
df_results = None  # Will be filled with cv scores
for i in tqdm(range(20), desc="Run", leave=False):
    for df_cv_scenarios, owner, impostors in tqdm(
        utils_generate_cv_scenarios(
            df_train_test,
            samples_per_subject_train=P.samples_per_subject_train,
            samples_per_subject_test=P.samples_per_subject_test,
            limit_test_samples=1,  # Samples overall
            seed=SEED + i,  # Change seed for different runs
            scaler=P.scaler,
            scaler_global=P.scaler_global,
            scaler_scope=P.scaler_scope,
            feature_cols=P.feature_cols,
        ),
        desc="Owner",
        total=df_train_test["subject"].nunique(),
        leave=False,
    ):
        X = np.array(df_cv_scenarios["X"].values.tolist())
        X = X.reshape(X.shape[-3], -1)  # flatten windows
        y = df_cv_scenarios["label"].values
        train_test_cv = utils_create_cv_splits(df_cv_scenarios["mask"].values, SEED)

        model = OneClassSVM(kernel="rbf", nu=P.ocsvm_nu, gamma=P.ocsvm_gamma)

        scores = cross_validate(
            model,
            X,
            y,
            cv=train_test_cv,
            scoring={"eer": utils_eer_scorer},
            n_jobs=CORES,
            verbose=0,
            return_train_score=True,
        )
        df_score = pd.DataFrame(scores)
        df_score["owner"] = owner
        df_score["train_eer"] = df_score["train_eer"].abs()  # Revert scorer's signflip
        df_score["test_eer"] = df_score["test_eer"].abs()
        df_results = pd.concat([df_results, df_score], axis=0)

df_results.to_csv(OUTPUT_PATH / f"{P.name}_detect_delay_results.csv", index=False)
df_results.head()

**Load Results from "Detection Delay" evaluation & prepare for plotting:**

In [ ]:
df_results = pd.read_csv(OUTPUT_PATH / f"{P.name}_detect_delay_results.csv")
df_results["owner"] = df_results["owner"].astype(str)
df_plot = df_results.copy()

**Plot Expanding Mean EER and confidence interval:**

In [ ]:
utils_plot_detect_delay(df_plot, factor=P.window_size / P.frequency, xlim=160)
utils_save_plot(plt, REPORT_PATH / f"buech2019-ocsvm-{P.name.lower()}-detection-delay.pdf")